In [1]:
#*********************************************************************************************#
#                                      TEXTRANK ALGORITHM                                     #
# This is a personal project to use a graph based algorith to create an application that can  #
# summurize a bunch of articles to a few sentences. It is a Natural Language Processing app-. #
# The application is a Python implementation, and uses libraries such as Numpy, Pandas, NLTK  #
#                           @created by: Traore, Youssouf - Oct 2018                          #
#**********************************************************************************************

In [2]:
# As usual, I start by importing a few librairies, and also the csv files that is saved in the same 
# folder as the py file. The article contains around five articles, structures as: article_id,
# artitle_text, source(which is a link).

In [3]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re

ModuleNotFoundError: No module named 'numpy'

In [ ]:
articlesdf = pd.read_csv("reading_articles_v4.csv")

In [ ]:
# Let's make sure we have the right file, by viewing the head of the data frame.

In [ ]:
articlesdf.head()

In [ ]:
# A quick viewing of an article using its index.

In [ ]:
articlesdf['article_text'][2]

In [ ]:
# Tokenization of sentences 

In [ ]:
from nltk.tokenize import sent_tokenize

sentences = []
for s in articlesdf['article_text']:
    sentences.append(sent_tokenize(s))
    
sentences = [y for x in sentences for y in x]

In [ ]:
# Let's try it

In [ ]:
sentences[:2]

In [ ]:
# Unzip GloVe from Stanford U.
!unzip glove.6B*.zip

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word]= coefs
f.close()

In [ ]:
len(word_embeddings)

In [ ]:
# Let's remove punctuations, numbers, and other 

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# Lowering the cases :)

clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
# Let's take care of Too commonly used words like: the...

nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function for stopwords

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
# Words vectors

In [ ]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


In [ ]:
# Creating vectors for our sentences

sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
            v = np.zeros((100,))
    sentence_vectors.append(v)

In [ ]:
# Using the cosine similarity approach

sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Initialize the matrix

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
import networkx as nx

In [ ]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i, s in enumerate(sentences)), reverse=True)

In [ ]:
# Extract the top 10 sentences as the summary

In [ ]:
for i in range(10):
    print(ranked_sentences[i][1])